# Utilización de pipelines

La documentación oficial se puede encontrar en: https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html

## Utilización de Pipeline con el conjunto entero

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA

from sklearn.metrics import accuracy_score

In [ ]:
# Cargar el conjunto de datos Iris
iris = load_iris(as_frame=True).frame
X = iris.drop(columns='target', axis=1)
y = iris['target']

In [ ]:
# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Crear un objeto Pipeline con un escalador y un clasificador SVM
pipeline = Pipeline([
    ('pca', PCA(n_components=2)), #no lo hemos visto, pero es un método de reducción de dimensionalidad
    ('scaler', StandardScaler())          
])

In [ ]:
pipeline.fit(X_train)

pipeline.transform(X_train)
pipeline.transform(X_test)

## Creación de tus propios transformadores

In [ ]:
# Cargar el conjunto de datos Iris
iris = load_iris(as_frame=True).frame
X = iris.drop(columns='target', axis=1)
y = iris['target']

#aniado columna categórica
X['categorica'] = 'valor por defecto'

# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train.head(5)

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

# Definir un transformador personalizado
class Transformador1(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):

        # Select only the numeric columns
        self.numeric_columns = list(X.dtypes[X.dtypes != 'object'].index)
        return self

    def transform(self, X):
        return X[self.numeric_columns]  # Seleccionar solo las columnas numéricas


In [ ]:
# Definir un transformador personalizado
class Transformador2(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X *5  # Multiplicar por 5

In [ ]:
#haremos pruebas con transformadores

t1 = Transformador1()
t1.fit_transform(X_train)
X_test = t1.transform(X_test)

X_test.head(5)

In [ ]:
# Crear el Pipeline con el transformador personalizado y el clasificador SVM
pipeline_with_transformer = Pipeline([
    ('transformador1', Transformador1()),       # Paso 1: Transformador personalizado
    ('transformador2', Transformador2()),       # Paso 2: Transformador personalizado
    ('scaler', StandardScaler()),               # Paso 3: Escalar los datos                    
])

In [ ]:
# Entrenar el modelo usando el conjunto de entrenamiento
pipeline_with_transformer.fit(X_train, y_train)
pipeline_with_transformer.transform(X_train)

### Transformadores con parámetros

También se le pueden pasar parámetros al transformador a la hora de constuirlo. Para ello, tendremos un método llamado ```__init__(self, <lista_parametros>)```. En el siguiente ejemplo, se pasa como parámetro la lista de columnas que quiero mantener en el dataset. 

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import LabelEncoder

class FiltroColumnas(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.columns]


A la hora de constuirlo le pasaré el parámetro. 

In [ ]:
columnas = ['sepal length (cm)', 'sepal width (cm)']

# Crear el Pipeline con el transformador personalizado y el clasificador SVM
pipeline_with_transformer = Pipeline([
    ('transformador1', Transformador1()),                       # Paso 1: Elimino las categóricas
    ('filtroColumnas', FiltroColumnas(columns=columnas)),       # Paso 2: Me quedo con las características del sepalo
    ('transformador2', Transformador2()),                       # Paso 2: Multiplico por 5 los valores
    ('scaler', StandardScaler())                                # Paso 3: Escalo los datos                      
])

In [ ]:
# Cargar el conjunto de datos Iris
iris = load_iris(as_frame=True).frame
X = iris.drop(columns='target', axis=1)
y = iris['target']

# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Entrenar el modelo usando el conjunto de entrenamiento
pipeline_with_transformer.fit(X_train, y_train)
pipeline_with_transformer.transform(X_train)

# Utilización de varios pipelines con varias columnas (NO ENTRA)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [ ]:
# Cargar datos
url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
titanic_data = pd.read_csv(url)
titanic_data.head()

Observaciones sobre las columnas:

* PassengerId es de tipo entero e identifica de manera única a cada pasajero, pero no será útil para nuestro modelo.
* Survived es el atributo objetivo de tipo booleano.
* Pclass clasifica a los pasajeros en clases. Aunque el valor es de tipo entero, esta característica es de naturaleza categórica.
* Name es de tipo objeto (cadena de texto). Probablemente sea único para cada pasajero y puede descartarse antes del entrenamiento.
* Sex es de tipo objeto (cadena de texto). Es de naturaleza categórica.
* Age es de tipo flotante. Al observar el resumen del conjunto de datos, podemos notar que contiene algunos valores faltantes.
* SibSp es de tipo entero y describe el número de hermanos/cónyuges de cada pasajero a bordo.
* Parch es de tipo entero y describe el número de padres/hijos de cada pasajero a bordo.
* Ticket es de tipo objeto (cadena de texto). Puede tener muchos (o todos) valores únicos.
* Fare es de tipo flotante.
* Cabin es de tipo objeto (cadena de texto). Es de naturaleza categórica pero contiene muchos valores faltantes.
* Embarked es de tipo objeto (cadena de texto). Es de naturaleza categórica y contiene algunos valores faltantes

In [ ]:
# Dividir los datos en características (X) y la variable objetivo (y)
X = titanic_data.drop("Survived", axis=1)
y = titanic_data["Survived"]

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Definir las columnas numéricas y categóricas
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

In [ ]:
# Crear transformers para los datos numéricos y categóricos
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

In [ ]:
# Combinar transformers en un preprocesador utilizando ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [ ]:
# Crear el pipeline final con el preprocesador y el modelo RandomForest
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                            ('classifier', RandomForestClassifier(random_state=42))])

In [ ]:
# Entrenar el modelo usando el pipeline
pipeline.fit(X_train, y_train)

In [ ]:
# Evaluar el modelo en el conjunto de prueba
accuracy = pipeline.score(X_test, y_test)
print(f"Accuracy: {accuracy}")